In [32]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "/home/jovyan/work/phd/topics/datasets/gtexrandom";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
allowWGCNAThreads()
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/cancers/unified_lung"

Allowing multi-threading with up to 2 threads.


In [33]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [34]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = "")) # 0, eng, Name
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [35]:
log2_1 <- function(x){
    log2(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log2_1))
dimnames(datExpr)[[1]] <- samples

In [36]:
datExpr[1:10,1:5]

,ENSG00000229344.1,ENSG00000198744.5,ENSG00000187730.8,ENSG00000169717.6,ENSG00000162592.9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-PLZ4-1226-SM-2I5FE,5.392317,5.672425,0.000000,0,0
GTEX-1EMGI-2726-SM-7P8S4,6.857981,7.754888,0.000000,0,0
GTEX-1C2JI-0008-SM-EXOHZ,3.584963,2.321928,0.000000,0,0
GTEX-1S5ZU-2526-SM-E6CIF,2.321928,2.807355,0.000000,0,0
GTEX-11EMC-3226-SM-5EGKW,5.357552,7.467606,5.930737,0,0
GTEX-11ZUS-2926-SM-5FQSL,5.169925,6.129283,6.894818,0,0
GTEX-18465-0226-SM-7LG5M,4.807355,5.392317,0.000000,0,0
GTEX-1I1GU-0926-SM-ACKVS,3.584963,4.523562,0.000000,0,0
GTEX-1JMLX-2326-SM-CXZJP,2.000000,2.584963,0.000000,0,0


In [37]:
head(allTraits)

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>
1,GTEX-1117F-0003-SM-58Q7G,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,GTEX-1117F-0003-SM-5DWSB,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,GTEX-1117F-0003-SM-6WBT7,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,GTEX-1117F-0011-R10a-SM-AHZ7F,NA,"B1, A1",,NA,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,GTEX-1117F-0011-R10b-SM-CYKQ8,NA,"B1, A1",,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,⋯,14648800,11999300,0.00315785,14669500,50.0354,0.00310538,0.99474,NA,0,50.1944


In [39]:
#labels <- c('Subtype_Selected','Subtype_Selected_Lum')
#labels <- c('cancer.type','Subtype_Selected')
#labels <- c('primary_sites', 'tissue_hd')
#labels <-c("Type","Subtype")
labels<-c("SMTS","SMTSD")
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$SAMPID #<--CHECK
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-PLZ4-1226-SM-2I5FE,Thyroid,Thyroid
GTEX-1EMGI-2726-SM-7P8S4,Colon,Colon - Transverse
GTEX-1C2JI-0008-SM-EXOHZ,Skin,Cells - Cultured fibroblasts
GTEX-1S5ZU-2526-SM-E6CIF,Skin,Skin - Not Sun Exposed (Suprapubic)
GTEX-11EMC-3226-SM-5EGKW,Brain,Brain - Cortex
GTEX-11ZUS-2926-SM-5FQSL,Brain,Brain - Cortex


In [40]:
table( dimnames(datExpr)[[1]]==datTraits$SMTS)


FALSE 
 1000 

In [41]:
y = datTraits$SMTS

In [42]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [43]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [44]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
pdf("wgcna_nsamplehier.pdf")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [45]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 450, minSize = 10)
table(clust)

clust
   1 
1000 

In [46]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [47]:
head(datExpr)

,ENSG00000229344.1,ENSG00000198744.5,ENSG00000187730.8,ENSG00000169717.6,ENSG00000162592.9,ENSG00000235054.3,ENSG00000228750.6,ENSG00000232208.2,ENSG00000235263.1,ENSG00000171819.4,⋯,ENSG00000210107.1,ENSG00000210112.1,ENSG00000210127.1,ENSG00000210144.1,ENSG00000210154.1,ENSG00000210156.1,ENSG00000210176.1,ENSG00000210191.1,ENSG00000210195.2,ENSG00000210196.2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-PLZ4-1226-SM-2I5FE,5.392317,5.672425,0.000000,0,0,0,0,0,0,0.000000,⋯,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
GTEX-1EMGI-2726-SM-7P8S4,6.857981,7.754888,0.000000,0,0,0,0,0,0,6.247928,⋯,3.169925,3.459432,1.584963,2.000000,0.000000,1.584963,2.584963,0.000000,1.584963,0.000000
GTEX-1C2JI-0008-SM-EXOHZ,3.584963,2.321928,0.000000,0,0,0,0,0,0,0.000000,⋯,0.000000,0.000000,0.000000,2.584963,0.000000,0.000000,1.584963,0.000000,1.000000,1.000000
GTEX-1S5ZU-2526-SM-E6CIF,2.321928,2.807355,0.000000,0,0,0,0,0,0,3.000000,⋯,0.000000,0.000000,1.584963,3.584963,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
GTEX-11EMC-3226-SM-5EGKW,5.357552,7.467606,5.930737,0,0,0,0,0,0,1.584963,⋯,1.000000,4.169925,2.321928,3.906891,3.459432,3.807355,2.807355,3.000000,2.000000,3.321928
GTEX-11ZUS-2926-SM-5FQSL,5.169925,6.129283,6.894818,0,0,0,0,0,0,0.000000,⋯,0.000000,4.247928,1.000000,2.584963,6.044394,2.584963,2.321928,1.584963,2.000000,1.000000


In [48]:
rownames(allTraits) <- allTraits$SAMPID#<--check
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-PLZ4-1226-SM-2I5FE,Thyroid,Thyroid
GTEX-1EMGI-2726-SM-7P8S4,Colon,Colon - Transverse
GTEX-1C2JI-0008-SM-EXOHZ,Skin,Cells - Cultured fibroblasts
GTEX-1S5ZU-2526-SM-E6CIF,Skin,Skin - Not Sun Exposed (Suprapubic)
GTEX-11EMC-3226-SM-5EGKW,Brain,Brain - Cortex
GTEX-11ZUS-2926-SM-5FQSL,Brain,Brain - Cortex


In [49]:
collectGarbage();

In [50]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("wgcna_samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

png 
  2

## 2

In [51]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=50, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("wgcna_scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 3000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3000 of 3000
   Power SFT.R.sq  slope truncated.R.sq mean.k. median.k. max.k.
1      1   0.1690 -0.742          0.300   773.0   501.000   1430
2      2   0.1120 -0.551          0.881   585.0   152.000   1280
3      3   0.0439 -0.317          0.968   515.0    92.700   1210
4      4   0.0536 -0.315          0.982   469.0    68.100   1160
5      5   0.0643 -0.318          0.985   433.0    57.000   1110
6      6   0.0776 -0.321          0.984   403.0    47.200   1070
7      7   0.0940 -0.324          0.992   377.0    39.900   1040
8      8   0.1150 -0.337          0.989   354.0    34.300   1000
9      9   0.1340 -0.339          0.986   334.0    29.600    972
10    10   0.1540 -0.338          0.978   315.0    25.700    943
11    12   0.1850 -0.354          0.954   283.0    19.700    889
12    14   0.2590 -0.368          0.936   256.0    15.200    841


png 
  2

In [52]:
net = blockwiseModules(datExpr, power = 10,
TOMType = "unsigned", minModuleSize = 5,
reassignThreshold = 0, mergeCutHeight = 0.4,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will use 2 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
     ..removing 1 genes from module 33 because their KME is too low.
 ..merging modules that are too close..
     mergeCloseModules: Merging modules whose distance is less than 0.4
       Calculating new MEs...


In [53]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("wgcna_netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [54]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [ ]:
#datTraits$cases.0.project.primary_site <- match(datTraits$cases.0.project.primary_site, unique(datTraits$cases.0.project.primary_site))
#datTraits$cases.0.project.disease_type <- match(datTraits$cases.0.project.disease_type, unique(datTraits$cases.0.project.disease_type))

In [55]:
datTraits$SMTS <- match(datTraits$SMTS, unique(datTraits$SMTS))
datTraits$SMTSD <- match(datTraits$SMTSD, unique(datTraits$SMTSD))

In [ ]:
#datTraits$cancer.type <- match(datTraits$cancer.type, unique(datTraits$cancer.type))
datTraits$Subtype_Selected <- match(datTraits$Subtype_Selected, unique(datTraits$Subtype_Selected))
datTraits$Subtype_Selected_Lum <- match(datTraits$Subtype_Selected_Lum, unique(datTraits$Subtype_Selected_Lum))

In [23]:
#datTraits$cancer.type <- match(datTraits$cancer.type, unique(datTraits$cancer.type))
datTraits$Type <- match(datTraits$Type, unique(datTraits$Type))
datTraits$Subtype <- match(datTraits$Subtype, unique(datTraits$Subtype))

In [57]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [58]:
pdf("wgcna_moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [59]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(rownames(datTraits));
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

Warning message in storage.mode(y) <- "double":
“NAs introduced by coercion”


In [60]:
module = "gray"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("wgcna_modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

ERROR: Error in abs(geneModuleMembership[moduleGenes, column]): non-numeric argument to mathematical function


In [ ]:
# Create the starting data frame
geneInfo0 = data.frame(
moduleColor = moduleColors,
geneTraitSignificance,
GSPvalue)
# Order modules by their significance for weight
modOrder = order(-abs(cor(MEs, weight, use = "p")));
# Add module membership information in the chosen order
for (mod in 1:ncol(geneModuleMembership))
{
oldNames = names(geneInfo0)
geneInfo0 = data.frame(geneInfo0, geneModuleMembership[, modOrder[mod]],
MMPvalue[, modOrder[mod]]);
names(geneInfo0) = c(oldNames, paste("MM.", modNames[modOrder[mod]], sep=""),
paste("p.MM.", modNames[modOrder[mod]], sep=""))
}
# Order the genes in the geneInfo variable first by module color, then by geneTraitSignificance
geneOrder = order(geneInfo0$moduleColor, -abs(geneInfo0$GS.weight));
geneInfo = geneInfo0[geneOrder, ]
write.csv(geneInfo, file = "geneInfo.csv")

In [ ]:
# Read in the probe annotation
annot = read.csv(file = "geneInfo.csv");
# Match probes in the data set to the probe IDs in the annotation file
probes = names(datExpr)
allLLIDs = probes
# $ Choose interesting modules
intModules = c("greenyellow", "darkorange", "darkgreen")
for (module in intModules)
{
# Select module probes
modGenes = (moduleColors==module)
# Get their entrez ID codes
modLLIDs = probes[modGenes];
# Write them into a file
fileName = paste("LocusLinkIDs-", module, ".txt", sep="");
write.table(as.data.frame(modLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)
}
# As background in the enrichment analysis, we will use all probes in the analysis.
fileName = paste("LocusLinkIDs-all.txt", sep="");
write.table(as.data.frame(allLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)

In [ ]:
require("biomaRt")
mart <- useMart("ENSEMBL_MART_ENSEMBL")
mart <- useDataset("hsapiens_gene_ensembl", mart)

ens <- allLLIDs
ensLookup <- gsub("\\.[0-9]*$", "", ens)

annotLookup <- getBM(
  mart=mart,
  attributes=c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"),
  filter="ensembl_gene_id",
  values=ensLookup,
  uniqueRows=TRUE)

annotLookup <- data.frame(
  ens[match(annotLookup$ensembl_gene_id, ensLookup)],
  annotLookup)

colnames(annotLookup) <- c(
  "original_id",
  c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"))

annotLookup

## 5 

In [61]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("wgcna_nethier.pdf", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

png 
  2

In [ ]:
rownames(allTraits)=allTraits$file_name
unique(allTraits[rownames(datTraits),'Subtype_Selected'])

In [ ]:
pdf("wgcna_nethier_legend.pdf", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'Subtype_Selected']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

In [ ]:
collectGarbage()

In [ ]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.4, minSize = 10)
table(clust)

In [ ]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

In [ ]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

In [ ]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

In [ ]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

In [ ]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

In [63]:
write.csv(cutree(sampleTree3, k=2), "wgcna/wgcna_level_5_labels.csv")
write.csv(cutree(sampleTree3, k=4), "wgcna/wgcna_level_4_labels.csv")
write.csv(cutree(sampleTree3, k=12), "wgcna/wgcna_level_3_labels.csv")
write.csv(cutree(sampleTree3, k=33), "wgcna/wgcna_level_2_labels.csv")
write.csv(cutree(sampleTree3, k=208), "wgcna/wgcna_level_1_labels.csv")
write.csv(cutree(sampleTree3, k=949), "wgcna/wgcna_level_0_labels.csv")

In [64]:
write.csv(geneModuleMembership, "wgcna/wgcna_level_0_word-dist.csv")

In [65]:
write.csv(MEs, "wgcna/wgcna_level_0_topic-dist.csv")